# B - Implement LSTMs

**Note:** The only `TODO`s in this exercise is within the `RNN` class. Run the remaining code to load data, train and evaluate the model. 

We can treat our task as a sequence classification task and use _Recurrent Neural Networks_. In particular, you will implement an RNN with LSTM cells. Here, we will consider the first 4,000 examples and only consider the 35 time-varying variables.

In [ ]:
#@title Run this cell to download preprocessed data (features + labels). { display-mode: "form" }
!pip install -U wget
!rm -rf preprocessed
!mkdir -p preprocessed
!mkdir -p checkpoint

import wget
wget.download('https://github.com/shengpu1126/BDSI2019-ML/raw/master/preprocessed/data_seq.npz', 'preprocessed/data_seq.npz')
wget.download('https://github.com/shengpu1126/BDSI2019-ML/raw/master/lib/prepare_data.py', 'lib/prepare_data.py')

In [ ]:
# GPU support
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device:', device)

In [ ]:
import os, time, random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from matplotlib import pyplot as plt
import itertools

## (a) Data preprocessing

The features have been preprocessed for you by running `prepare_data.py`. 

Since the raw data for each patient are irregularly-sampled time-series data, we will first resample them to have even intervals. For the 48-hour period, we group the observations into 2-hour windows so there are 24 time windows. Within each window, we compute the mean value and missing flag for each of the 35 variables. We then use a simple forward imputation approach to estimate some missing values using df.ffill(). This copies an observed value to future unobserved timesteps. All of these preprocessing steps are implemented in generate features(df). The output of this func- tion is a pd.DataFrame with 70 feature columns and 24 rows, one for each 2-hour window. See sample features.csv for an example of the features extracted for one patient.

In the `main` function of `prepare_data.py`, we first call `generate_features(df)` on every patient. We then call `impute_missing_values()` to impute the remaining missing values with population mean, and `standardize_features()` to standardize the data to mean 0 and std 1. These functions are very similar to what you did during Week 1. We have already run `prepare_data.py` to preprocess the raw csv files into features and labels, and the output is saved as `data_seq.npz` (provided).

In [ ]:
class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]).float(), torch.tensor([self.y[idx]]).float()
    def __len__(self):
        return len(self.X)

def get_train_val_test(batch_size=64):
    f = np.load('preprocessed/data_seq.npz')
    X, y = f['X'], f['y']
    print(X.shape, y.shape)
    
    print('Creating splits')
    Xtr, X__, ytr, y__ = train_test_split(X,   y,   train_size=0.8, stratify=y,   random_state=0)
    Xva, Xte, yva, yte = train_test_split(X__, y__, test_size=0.5, stratify=y__, random_state=0)
    
    tr = SimpleDataset(Xtr, ytr)
    va = SimpleDataset(Xva, yva)
    te = SimpleDataset(Xte, yte)
    
    tr_loader = DataLoader(tr, batch_size=batch_size, shuffle=True)
    va_loader = DataLoader(va, batch_size=batch_size)
    te_loader = DataLoader(te, batch_size=batch_size)
    
    print('Feature shape, Label shape, Class balance:')
    print('\t', tr_loader.dataset.X.shape, tr_loader.dataset.y.shape, tr_loader.dataset.y.mean())
    print('\t', va_loader.dataset.X.shape, va_loader.dataset.y.shape, va_loader.dataset.y.mean())
    print('\t', te_loader.dataset.X.shape, te_loader.dataset.y.shape, te_loader.dataset.y.mean())
    return tr_loader, va_loader, te_loader

In [ ]:
tr_loader, va_loader, te_loader = get_train_val_test(batch_size=64)

## (b) RNN architecture
Study the architecture below. How many learnable parameters are there? Treat biases associated with input and hidden state as different parameters.

![](lib/lstm.png)

## (c) Implementing an LSTM network

Complete the `RNN` model class by implementing the methods `__init()__`, `init_weights()`, `init_hidden(x)` and `forward(x)`. Note that unlike simple feed-forward neural networks, the `forward` function will contain a for-loop since we are dealing with sequential data.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = ???
        self.fc = ???
        self.init_weights()

    def init_weights(self):
        # TODO: Initialize weights and bias to the specified distributions
        ???
    
    def forward(self, x):
        N, T, d = x.shape
        
        # Initialize hidden states h_0, c_0
        h_t, c_t = self.init_hidden(x)
        
        # Loop through the time-steps
        for ???:
            # Calculate one time-step
            ???
        
        # Take the output from the final time-step, pass into the fully-connected layer
        # Apply output activation
        z = ???
        return z
    
    def init_hidden(self, x):
        h_0 = ???
        c_0 = ???
        return h_0, c_0

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = RNN(70, 64, 1)
print('Number of float-valued parameters:', count_parameters(model))

## (d) Train the LSTM

The data are split into train (80%), validation (10%) and test (10%). Run the following code to train the model and display the resulting training plots (for loss and AUROC). At which epoch do we start overfitting?

In [ ]:
def _train_epoch(data_loader, model, criterion, optimizer):
    """
    Train the `model` for one epoch of data from `data_loader`
    Use `optimizer` to optimize the specified `criterion`
    """
    model.train()
    for i, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)
        
        # clear parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

def _evaluate_epoch(tr_loader, va_loader, model, criterion):
    model.eval()
    with torch.no_grad():
        # Evaluate on train
        y_true, y_score = [], []
        running_loss = []
        for X, y in tr_loader:
            X, y = X.to(device), y.to(device)
            output = model(X)
            y_true.append(y.cpu().numpy())
            y_score.append(output.cpu().numpy())
            running_loss.append(criterion(output, y).item())

        y_true, y_score = np.concatenate(y_true), np.concatenate(y_score)
        train_loss = np.mean(running_loss)
        train_score = metrics.roc_auc_score(y_true, y_score)
        print('tr loss', train_loss, 'tr AUROC', train_score)

        # Evaluate on validation
        y_true, y_score = [], []
        running_loss = []
        for X, y in va_loader:
            X, y = X.to(device), y.to(device)
            with torch.no_grad():
                output = model(X)
                y_true.append(y.cpu().numpy())
                y_score.append(output.cpu().numpy())
                running_loss.append(criterion(output, y).item())

        y_true, y_score = np.concatenate(y_true), np.concatenate(y_score)
        val_loss = np.mean(running_loss)
        val_score = metrics.roc_auc_score(y_true, y_score)
        print('va loss', val_loss, 'va AUROC', val_score)
    return train_loss, val_loss, train_score, val_score

def save_checkpoint(model, epoch, checkpoint_dir):
    state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
    }

    filename = os.path.join(checkpoint_dir, 'epoch={}.checkpoint.pth.tar'.format(epoch))
    torch.save(state, filename)

In [ ]:
tr_loader, va_loader, te_loader = get_train_val_test(batch_size=64)
    
torch.random.manual_seed(0)
np.random.seed(0)
random.seed(0)

n_epochs = 30
learning_rate = 1e-3

model = RNN(70, 64, 1)
print('Number of float-valued parameters:', count_parameters(model))

model = model.to(device)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

outputs = []

print('Epoch', 0)
out = _evaluate_epoch(tr_loader, va_loader, model, criterion)
outputs.append(out)

for epoch in range(0, n_epochs):
    print('Epoch', epoch+1)
    # Train model
    _train_epoch(tr_loader, model, criterion, optimizer)

    # Evaluate model
    out = _evaluate_epoch(tr_loader, va_loader, model, criterion)
    outputs.append(out)

    # Save model parameters
    save_checkpoint(model, epoch+1, 'checkpoint/')

train_losses, val_losses, train_scores, val_scores = zip(*outputs)

fig, ax = plt.subplots(figsize=(5,5))
plt.plot(range(n_epochs + 1), train_scores, '--o', label='Train')
plt.plot(range(n_epochs + 1), val_scores, '--o', label='Validation')
plt.xlabel('epoch')
plt.ylabel('AUROC')
plt.legend()
plt.savefig('auroc.png', dpi=300)

fig, ax = plt.subplots(figsize=(5,5))
plt.plot(range(n_epochs + 1), train_losses, '--o', label='Train')
plt.plot(range(n_epochs + 1), val_losses, '--o', label='Validation')
plt.xlabel('epoch')
plt.ylabel('Loss (binary cross entropy)')
plt.legend()
plt.savefig('loss.png', dpi=300)


## (e) Evaluation on test set

Run the following code to evaluate the model on the test set. Choose the epoch number to load the checkpoint from. Report the final model performance on the test set (AUROC score).

In [ ]:
def restore_checkpoint(model, checkpoint_dir, cuda=False):
    """
    If a checkpoint exists, restores the PyTorch model from the checkpoint.
    Returns the model and the current epoch.
    """
    cp_files = [file_ for file_ in os.listdir(checkpoint_dir)
        if file_.startswith('epoch=') and file_.endswith('.checkpoint.pth.tar')]

    if not cp_files:
        print('No saved model parameters found')
        if force:
            raise Exception("Checkpoint not found")
        else:
            return model, 0, []
    
    # Find latest epoch
    for i in itertools.count(1):
        if 'epoch={}.checkpoint.pth.tar'.format(i) in cp_files:
            epoch = i
        else:
            break

    print("Which epoch to load from? Choose in range [1, {}].".format(epoch))
    inp_epoch = int(input())
    if inp_epoch not in range(1, epoch+1):
        raise Exception("Invalid epoch number")

    filename = os.path.join(checkpoint_dir,
        'epoch={}.checkpoint.pth.tar'.format(inp_epoch))

    print("Loading from checkpoint {}".format(filename))
    
    if cuda:
        checkpoint = torch.load(filename)
    else:
        # Load GPU model on CPU
        checkpoint = torch.load(filename,
            map_location=lambda storage, loc: storage)

    try:
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        print("=> Successfully restored checkpoint (trained for {} epochs)"
            .format(checkpoint['epoch']))
    except:
        print("=> Checkpoint not successfully restored")
        raise

    return model, inp_epoch

def _evaluate_epoch(data_loader, model, criterion):
    model.eval()
    with torch.no_grad():
        y_true, y_score = [], []
        running_loss = []
        for X, y in data_loader:
            output = model(X)
            y_true.append(y.numpy())
            y_score.append(output)
            running_loss.append(criterion(output, y).item())
        y_true, y_score = np.concatenate(y_true), np.concatenate(y_score)
    
    loss = np.mean(running_loss)
    score = metrics.roc_auc_score(y_true, y_score)
    return loss, score

_, _, te_loader = get_train_val_test(batch_size=64)
model = RNN(70, 64, 1)
model, _ = restore_checkpoint(model, 'checkpoint/')
criterion = torch.nn.BCELoss()
loss, score = _evaluate_epoch(te_loader, model, criterion)
print('Test loss :', loss)
print('Test AUROC:', score)

## (f) Post-mortem / reflection:
1. What hyperparameters are associated with this modeling approach? List at least 2.
2. Here, we focused on only time-varying data. How might you incorporate the time-invariant data (e.g. age, gender)? Answer is not unique.